In [1]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from googletrans import Translator
from transformers import Trainer, TrainingArguments, T5Tokenizer, T5ForConditionalGeneration

device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

## **Prueba traductor**

In [2]:
trans = Translator()
print(trans.translate('Pues hazme los deberes, furcia barata!').text)

Well, do my homework for me, you cheap whore!


## **Dataset**

In [3]:
df = pd.DataFrame(pd.read_csv('3_joined_dataset.csv'))
train_dataframe, val_dataframe = train_test_split(df, train_size=0.8, random_state=42) # random_state=42 for reproducibility

train_toxic_texts = list(train_dataframe['toxic_sentence'])
train_neutral_texts = list(train_dataframe['neutral_sentence'])

val_toxic_texts = list(val_dataframe['toxic_sentence'])
val_neutral_texts = list(val_dataframe['neutral_sentence'])

## **Detoxificador**

In [4]:
model_name = "google/flan-t5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
model.config.task_specific_params = {
    "neutralization": {
        "early_stopping": True,
        "length_penalty": 1.0,
        "max_length": 100,
        "min_length": 10,
        "no_repeat_ngram_size": 2,
        "num_beams": 5,
        "prefix": "neutralize: "
    }
}

In [ ]:
# Congelar todos los parámetros del modelo
for param in model.parameters():
    param.requires_grad = False

# Descongelar las últimas 3 capas del decoder
for param in model.decoder.block[-3:].parameters():
    param.requires_grad = True

# Mantener la capa de salida (`lm_head`) entrenable
for param in model.lm_head.parameters():
    param.requires_grad = True

# model.encoder.embed_tokens.weight.requires_grad = True
# model.decoder.embed_tokens.weight.requires_grad = True

total_params = sum(p.numel() for p in model.parameters())
trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f"Total de parámetros: {total_params:,}")
print(f"Parámetros entrenables: {trainable_params:,}")
print(f"Parámetros congelados: {total_params - trainable_params:,}")

Total de parámetros: 76,961,152
Parámetros entrenables: 25,891,328
Parámetros congelados: 51,069,824


In [ ]:
class DetoxDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, target_encodings):
        self.encodings = encodings
        self.target_encodings = target_encodings

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: val[idx].clone().detach() for key, val in self.encodings.items()}
        item['labels'] = self.target_encodings['input_ids'][idx].clone().detach()
        return item

In [9]:
toxic_encodings_train = tokenizer(train_toxic_texts, truncation=True, padding=True, return_tensors="pt")
neutral_encodings_train = tokenizer(train_neutral_texts, truncation=True, padding=True, return_tensors="pt")

toxic_encodings_val = tokenizer(val_toxic_texts, truncation=True, padding=True, return_tensors="pt")
neutral_encodings_val = tokenizer(val_neutral_texts, truncation=True, padding=True, return_tensors="pt")

train_dataset = DetoxDataset(toxic_encodings_train, neutral_encodings_train)
val_dataset = DetoxDataset(toxic_encodings_val, neutral_encodings_val)

training_args = TrainingArguments(
    output_dir='./checkpoints',
    # evaluation_strategy="no",
    # evaluation_strategy="epoch",
    evaluation_strategy="steps",
    eval_steps=146*5, # cada 5 epochs, teniendo batches de 128
    per_device_train_batch_size=128,
    per_device_eval_batch_size=128,
    learning_rate=2e-4,
    num_train_epochs=25,
    report_to=["none"], # para no pedir login de 'wandb' y otros
    # fp16=True, # acelera el entrenaminento pero lo empeora  
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

In [10]:
trainer.train()
# trainer.train(resume_from_checkpoint="checkpoints/checkpoint-3000") # da error por archivos faltantes

  0%|          | 0/3650 [00:00<?, ?it/s]

{'loss': 1.9358, 'grad_norm': 0.0750923678278923, 'learning_rate': 0.00017260273972602742, 'epoch': 3.42}


  0%|          | 0/37 [00:00<?, ?it/s]

{'eval_loss': 0.1498144119977951, 'eval_runtime': 47.092, 'eval_samples_per_second': 99.146, 'eval_steps_per_second': 0.786, 'epoch': 5.0}
{'loss': 0.1665, 'grad_norm': 0.06426659226417542, 'learning_rate': 0.0001452054794520548, 'epoch': 6.85}


  0%|          | 0/37 [00:00<?, ?it/s]

...

  0%|          | 0/37 [00:00<?, ?it/s]

{'eval_loss': 0.14401212334632874, 'eval_runtime': 46.8799, 'eval_samples_per_second': 99.595, 'eval_steps_per_second': 0.789, 'epoch': 20.0}
{'loss': 0.145, 'grad_norm': 0.06218307092785835, 'learning_rate': 3.561643835616438e-05, 'epoch': 20.55}
{'loss': 0.1443, 'grad_norm': 0.08183270692825317, 'learning_rate': 8.21917808219178e-06, 'epoch': 23.97}


  0%|          | 0/37 [00:00<?, ?it/s]

{'eval_loss': 0.14377838373184204, 'eval_runtime': 47.1456, 'eval_samples_per_second': 99.034, 'eval_steps_per_second': 0.785, 'epoch': 25.0}
{'train_runtime': 1669.5538, 'train_samples_per_second': 279.611, 'train_steps_per_second': 2.186, 'train_loss': 0.39603337849656195, 'epoch': 25.0}


TrainOutput(global_step=3650, training_loss=0.39603337849656195, metrics={'train_runtime': 1669.5538, 'train_samples_per_second': 279.611, 'train_steps_per_second': 2.186, 'train_loss': 0.39603337849656195, 'epoch': 25.0})

## **GUARDAR Y CARGAR**

In [ ]:
save_directory = "./modelo_local"

model.save_pretrained(save_directory)
tokenizer.save_pretrained(save_directory)

model = T5ForConditionalGeneration.from_pretrained(save_directory)
tokenizer = T5Tokenizer.from_pretrained(save_directory)

('./modelo_local\\tokenizer_config.json',
 './modelo_local\\special_tokens_map.json',
 './modelo_local\\spiece.model',
 './modelo_local\\added_tokens.json')

## **PRUEBAS DETOX**

In [12]:
def detoxify_sentence(text: str):
    # Tokenizar la oración tóxica
    inputs: dict = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    # Mover los tensores a la GPU, si está disponible
    inputs = {key: val.to(device) for key, val in inputs.items()}

    outputs = model.generate(inputs["input_ids"], max_new_tokens=100)
    texto_neutralizado = tokenizer.decode(outputs[0], skip_special_tokens=True)

    return texto_neutralizado

In [19]:
test_sentences = [
    "Do my homework, cheap whore!",
    "shut your angry ass up",
    "You are a stupid person",
]

for sentence in test_sentences:
    neutral_sentence = detoxify_sentence(sentence)
    print(f"Original: {sentence} -> Neutral: {neutral_sentence}") 

Original: Do my homework, cheap whore! -> Neutral: Do my homework, cheap whore!
Original: shut your angry ass up -> Neutral: Stop talking.
Original: You are a stupid person -> Neutral: You are not smart


---

---

## **BUCLE DE TRADUCCIÓN** (26 mins)

In [23]:
csv = pd.read_csv('1_detox_dataset.csv')
df = pd.DataFrame(csv)

trans = Translator()
# print(trans.translate(df.iloc[400]['toxic_sentence'], dest='en').text)
print(trans.translate("ታምራት ነገራ ፅንፈኞችን ከላይ እስከታች ሰብስበህ ሁለት ሰንበት ፀበልና ስልጠና ስጥልን", dest='en').text)

Tamrat Negara gathered the extremists from top to bottom and gave us two Sabbath prayers and training.


---

In [ ]:
df = pd.DataFrame(pd.read_csv("1_detox_dataset.csv"))
df_trans = pd.DataFrame(columns=["toxic_sentence", "neutral_sentence"])
traductor = Translator()

for index in range(len(df)):
    toxic_translated = traductor.translate(df.iloc[index]['toxic_sentence'], dest='en').text
    neutral_translated = traductor.translate(df.iloc[index]['neutral_sentence'], dest='en').text
    df_trans.loc[len(df_trans)] = [toxic_translated, neutral_translated]
    break

df_trans.to_csv("2_detox_dataset_trans.csv", index=False)